# Load SAM checkpoint for image embeddings

In [ ]:
cd /home/icb/hanyi.zhang/Detection_Head/segment-anything

In [ ]:
from segment_anything import sam_model_registry, SamPredictor
from segment_anything.separate_sam_encoder import SamEncoder
import torch

sam_checkpoint = "/home/icb/hanyi.zhang/Detection_Head/segment-anything/sam_vit_l_0b3195.pth"
model_type = "vit_l"

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

sam_image_encoder = SamEncoder(sam, device)
predictor = SamPredictor(sam)

### Define necessary functions here

In [ ]:
import os
from PIL import Image
import numpy as np
import cv2
from copy import deepcopy
from typing import Tuple

def emb_images(image_folder, embed_folder, encoder):
    """
    Process images to obtain embeddings and save them.

    Parameters:
    - image_folder: str, path to the folder containing source images
    - embed_folder: str, path to the folder where embeddings will be saved
    - encoder: the encoder object that processes images to obtain embeddings
    """
    # Create the embedding directory if it doesn't exist
    os.makedirs(embed_folder, exist_ok=True)
    
    # List all files in the image directory
    images = os.listdir(image_folder)

    # Iterate over each image file
    for i, img in enumerate(images, start=1):
        if i % 100 == 0:
            print(f"Processed {i} images.")
        
        # Construct full path to the image file
        img_path = os.path.join(image_folder, img)
        
        # Load the image file and convert it to a numpy array
        with Image.open(img_path) as image:
            image = image.convert('RGB')
            img_np = np.array(image)
        
        # Process the image to get its embedding
        img_embed = encoder.set_image(img_np)
        
        # Replace the original extension with 'npy' for the output file
        embed_path = os.path.join(embed_folder, img.replace('.png', '.npy'))
        
        # Save the embedding to the specified path
        np.save(embed_path, img_embed.cpu().detach().numpy())

# Function to load checkpoint and perform evaluation on one example
def evaluate_single_example(model, img_emb):
    # Prepare the input
    img_emb = torch.tensor(img_emb, dtype=torch.float32).to(device)  # Convert and move to GPU if available
    
    # Dummy query embedding (you may adjust this based on your actual implementation)
    query_embed = model.query_embed.weight.to(device)
    
    # Get positional embedding
    pos_embedding = model.position_embedding(img_emb)
    
    # Forward pass
    with torch.no_grad():
        outputs = model(query_embedding=query_embed, image_embedding=img_emb, pos_embedding=pos_embedding)

    # Apply softmax to pred_logits
    pred_logits = torch.nn.functional.softmax(outputs['pred_logits'][0], dim=-1)
    
    # Convert softmax output to binary predictions (0 or 1)
    binary_predictions = (pred_logits[:, 0] > pred_logits[:, 1]).cpu().numpy().astype(int)
    
    # Filter out boxes with binary prediction value 0
    pred_boxes = outputs['pred_boxes'].cpu().numpy()[0]
    filtered_boxes = pred_boxes[binary_predictions == 1]
    confidence_scores = pred_logits[binary_predictions == 1, 0].cpu().numpy()
    
    return filtered_boxes, confidence_scores

def evaluate_single_example_given_threshold(model, img_emb, confidence_threshold):
    # Prepare the input
    img_emb = torch.tensor(img_emb, dtype=torch.float32).to(device)  # Convert and move to GPU if available
    
    # Dummy query embedding (you may adjust this based on your actual implementation)
    query_embed = model.query_embed.weight.to(device)
    
    # Get positional embedding
    pos_embedding = model.position_embedding(img_emb)
    
    # Forward pass
    with torch.no_grad():
        outputs = model(query_embedding=query_embed, image_embedding=img_emb, pos_embedding=pos_embedding)

    # Apply softmax to pred_logits
    pred_logits = torch.nn.functional.softmax(outputs['pred_logits'][0], dim=-1)
    
    # Convert softmax output to binary predictions (0 or 1) based on the confidence score threshold
    binary_predictions = (pred_logits[:, 0] > torch.tensor(confidence_threshold)).cpu().numpy().astype(int)
    
    # Filter out boxes with binary prediction value 0
    pred_boxes = outputs['pred_boxes'].cpu().numpy()[0]
    filtered_boxes = pred_boxes[binary_predictions == 1]
    confidence_scores = pred_logits[binary_predictions == 1, 0].cpu().numpy()
    
    return filtered_boxes, confidence_scores

def non_max_suppression(bboxes, scores, threshold):
    """Perform Non-Maximum Suppression (NMS) on bounding boxes.
    
    Args:
    bboxes (numpy.ndarray): Array of bounding boxes in the format (x1, y1, x2, y2).
    scores (numpy.ndarray): Array of scores for each bounding box.
    threshold (float): IoU threshold for suppression.

    Returns:
    numpy.ndarray: Array of indices of bounding boxes to keep.
    """
    # Sort the bounding boxes by the scores in descending order
    indices = np.argsort(scores)[::-1]
    
    keep = []
    while len(indices) > 0:
        current = indices[0]
        keep.append(current)
        
        if len(indices) == 1:
            break
        
        current_box = bboxes[current]
        remaining_boxes = bboxes[indices[1:]]
        
        # Compute IoU of the current box with the rest
        ious = np.array([compute_iou_box(current_box, box) for box in remaining_boxes])
        
        # Select boxes with IoU less than the threshold
        indices = indices[1:][ious < threshold]
    
    return np.array(keep)

def compute_iou_box(box1, box2):
    x1_max = max(box1[0], box2[0])
    y1_max = max(box1[1], box2[1])
    x2_min = min(box1[2], box2[2])
    y2_min = min(box1[3], box2[3])

    intersection_area = max(0, x2_min - x1_max) * max(0, y2_min - y1_max)

    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])

    union_area = box1_area + box2_area - intersection_area

    if union_area == 0:
        return 0.0

    iou = intersection_area / union_area
    return iou

# Define the transformation functions
def get_preprocess_shape(oldh: int, oldw: int, long_side_length: int) -> Tuple[int, int]:
    """
    Compute the output size given input size and target long side length.
    """
    scale = long_side_length * 1.0 / max(oldh, oldw)
    newh, neww = oldh * scale, oldw * scale
    neww = int(neww + 0.5)
    newh = int(newh + 0.5)
    return (newh, neww)

def inverse_coords(coords: np.ndarray, original_size: Tuple[int, int], target_length=1024) -> np.ndarray:
    """
    Inverse transformation of coordinates from resized back to original.
    """
    old_h, old_w = original_size
    new_h, new_w = get_preprocess_shape(old_h, old_w, target_length)
    coords = deepcopy(coords).astype(float)
    coords[..., 0] = coords[..., 0] * (old_w / new_w)
    coords[..., 1] = coords[..., 1] * (old_h / new_h)
    return coords

def inverse_boxes(boxes: np.ndarray, original_size: Tuple[int, int], target_length=1024) -> np.ndarray:
    """
    Inverse transformation of boxes from resized back to original.
    """
    boxes = inverse_coords(boxes.reshape(-1, 2, 2), original_size, target_length)
    return boxes.reshape(-1, 4)

def convert_boxes(boxes):
    # Convert the boxes from center format to [x_min, y_min, x_max, y_max] format
    converted_boxes = np.zeros_like(boxes)
    converted_boxes[:, 0] = boxes[:, 1] - boxes[:, 3] / 2.0  # x_min
    converted_boxes[:, 1] = boxes[:, 0] - boxes[:, 2] / 2.0  # y_min
    converted_boxes[:, 2] = boxes[:, 1] + boxes[:, 3] / 2.0  # x_max
    converted_boxes[:, 3] = boxes[:, 0] + boxes[:, 2] / 2.0  # y_max
    return converted_boxes

def transform_mask(mask):
    """
    Transforms a mask of shape (N, 1, H, W) into a single mask of shape (H, W),
    where background is 0 and different masks are labeled as 1, 2, 3, ..., N.
    If the mask contains only zeros, it returns a zero-filled mask of shape (H, W).
    """
    # Check if the mask is entirely zeros
    if len(mask.shape) == 2:
        return mask

    # Get the number of masks
    num_masks = mask.shape[0]
    
    # Initialize a new mask with zeros (background)
    transformed_mask = np.zeros((mask.shape[2], mask.shape[3]), dtype=np.uint8)

    # Assign label 1, 2, 3, ..., N to each mask respectively
    for i in range(num_masks):
        single_mask = mask[i, 0, :, :]  # Extract each mask
        transformed_mask[single_mask > 0] = i + 1  # Label the mask with 1, 2, ..., N

    return transformed_mask

### Please uncomment the following cell if image embeddings are unavailable.

In [ ]:
'''embed_folder = '/home/icb/hanyi.zhang/public_organoid_datasets/OrganoID_dataset/testing/images_sam_emb'
emb_images(images_path, embed_folder, sam_image_encoder)'''

# Load detection head checkpoint for predictions

In [ ]:
cd /home/icb/hanyi.zhang/Detection_Head

In [ ]:
# Load checkpoint here
from detection_head_model import DetectionHead
import torch

#checkpoint_path = '/home/icb/hanyi.zhang/Detection_Head/checkpoints/DetectionHead_finetuning-epoch=599-val_loss=7.82.ckpt'
#checkpoint_path = '/home/icb/hanyi.zhang/Detection_Head/checkpoints/DetectionHead_training-orga-epoch=599-val_loss=8.23.ckpt'
checkpoint_path = '/home/icb/hanyi.zhang/Detection_Head/checkpoints/DetectionHead_fine_tuning_combined-epoch=599-val_loss=7.38.ckpt'

model = DetectionHead.load_from_checkpoint(checkpoint_path)

# Set the model to evaluation mode
model.eval()

# Determine device (GPU or CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

In [ ]:
# Print the number of trainable parameters in Mio (millions)
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Number of trainable parameters: {trainable_params / 1e6:.2f} Mio")

# Predict

### Define paths to the folder containing images and image embeddings.

In [ ]:
import os
# Define paths
base_path = '/home/icb/hanyi.zhang/MouseOrganoids/testing_PDAC'

img_folder = base_path + '/images'
img_emb_folder = base_path + '/images_emb'

# Define paths to save the predicted masks and bboxes
predictions_save_path = base_path + '/pred_mask_th'
bboxes_save_path = base_path + '/pred_bbox_th'

# Define confidence score threshold
confidence_score = 0.5

In [ ]:
os.makedirs(predictions_save_path, exist_ok=True)
os.makedirs(bboxes_save_path, exist_ok=True)

print(img_folder)

# Get list of files
img_emb_files = os.listdir(img_emb_folder)

# NMS threshold
threshold = 0.5

for img_emb_file in img_emb_files:
    file_idx = img_emb_file.rsplit('.', 1)[0]
    print(file_idx)

    # Define image path, label path and image embedding path
    image_path = os.path.join(img_folder, file_idx + '.png')
    image_embedding_path = os.path.join(img_emb_folder, img_emb_file)
    mask_save_path = os.path.join(predictions_save_path, file_idx + '.npy')
    bbox_save_path = os.path.join(bboxes_save_path, file_idx + '.npy')

    # Read the image
    image = Image.open(image_path)

    # Get the original size of the image
    original_size = image.size  # (width, height)
    original_size = (original_size[1], original_size[0])  # Convert to (height, width)

    # Load image embedding
    image_embedding = np.load(image_embedding_path)

    # Predict boxes and convert to original size
    #predicted_boxes, confidence_scores = evaluate_single_example(model, image_embedding)
    predicted_boxes, confidence_scores = evaluate_single_example_given_threshold(model, image_embedding, confidence_score)
    original_boxes_pred = inverse_boxes(convert_boxes(predicted_boxes) * 1024, original_size)
        
    kept_indices = non_max_suppression(original_boxes_pred, confidence_scores, threshold)
    if len(kept_indices) != 0:
        original_boxes_pred_filtered = original_boxes_pred[kept_indices]
        filtered_confidences = confidence_scores[kept_indices]
        
        # Predict masks
        image = cv2.imread(image_path)
        predictor.set_image(image)
        input_boxes = torch.tensor(original_boxes_pred_filtered).to(device)
        transformed_boxes = predictor.transform.apply_boxes_torch(input_boxes, image.shape[:2])
        masks, _, _ = predictor.predict_torch(
            point_coords=None,
            point_labels=None,
            boxes=transformed_boxes,
            multimask_output=False,
        )
        predicted_masks = masks.cpu().numpy()
        predicted_masks = transform_mask(predicted_masks)

        # Save the bounding boxes
        np.save(bbox_save_path, original_boxes_pred_filtered)
    else:
        # If no boxes, create an all-zero mask and save empty bounding boxes
        predicted_masks = np.zeros((original_size[0], original_size[1]), dtype=np.uint8)
        np.save(bbox_save_path, np.array([]))  # Save empty array for bounding boxes

    # Save the mask
    np.save(mask_save_path, predicted_masks)



# Compute metric values

In [ ]:
import os
import numpy as np
from scipy.optimize import linear_sum_assignment
from skimage.measure import label

def compute_iou(pred_mask, gt_mask, pred_label, gt_label):
    pred_instance = (pred_mask == pred_label)
    gt_instance = (gt_mask == gt_label)
    
    intersection = np.logical_and(pred_instance, gt_instance).sum()
    union = np.logical_or(pred_instance, gt_instance).sum()
    
    if union == 0:
        return 0.0
    else:
        return intersection / union

def compute_metrics(pred_mask, gt_mask, iou_threshold=0.5):
    pred_labels = np.unique(pred_mask)
    gt_labels = np.unique(gt_mask)
    
    # Remove background label (0)
    pred_labels = pred_labels[pred_labels != 0]
    gt_labels = gt_labels[gt_labels != 0]
    
    num_preds = len(pred_labels)
    num_gts = len(gt_labels)
    
    # Create the IoU matrix
    iou_matrix = np.zeros((num_preds, num_gts))
    
    for i, p_label in enumerate(pred_labels):
        for j, g_label in enumerate(gt_labels):
            iou_matrix[i, j] = compute_iou(pred_mask, gt_mask, p_label, g_label)
    
    # Hungarian matching
    row_ind, col_ind = linear_sum_assignment(-iou_matrix)
    
    tp = []
    fp = []
    fn = []
    
    matched_gt = set()
    matched_pred = set()
    
    for i, j in zip(row_ind, col_ind):
        if iou_matrix[i, j] >= iou_threshold:
            tp.append(iou_matrix[i, j])
            matched_gt.add(j)
            matched_pred.add(i)
    
    for i in range(num_preds):
        if i not in matched_pred:
            fp.append(pred_labels[i])
    
    for j in range(num_gts):
        if j not in matched_gt:
            fn.append(gt_labels[j])
    
    # Calculate PQ
    pq = np.sum(tp) / (len(tp) + 0.5 * len(fp) + 0.5 * len(fn))
    
    # Calculate precision, recall, and F1-score
    precision = len(tp) / (len(tp) + len(fp)) if (len(tp) + len(fp)) > 0 else 0
    recall = len(tp) / (len(tp) + len(fn)) if (len(tp) + len(fn)) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    # Calculate mean IoU
    mean_iou = np.mean(tp) if len(tp) > 0 else 0
    
    # Calculate Dice coefficient
    dice_coefficient = np.mean([2 * iou / (1 + iou) for iou in tp]) if len(tp) > 0 else 0
    
    return len(tp), len(fp), len(fn), pq, precision, recall, f1_score, mean_iou, dice_coefficient

def process_folder(pred_folder, gt_folder, mask_need_transform, iou_threshold=0.5):
    pred_files = sorted(os.listdir(pred_folder))
    gt_files = sorted(os.listdir(gt_folder))
    
    pq_values = []
    precision_values = []
    recall_values = []
    f1_score_values = []
    mean_iou_values = []
    dice_values = []

    for i, (pred_file, gt_file) in enumerate(zip(pred_files, gt_files)):
        print(pred_file)
        pred_mask = np.load(os.path.join(pred_folder, pred_file))
        if mask_need_transform:
            pred_mask = transform_mask(pred_mask)
        gt_mask = np.load(os.path.join(gt_folder, gt_file))
        
        tp, fp, fn, pq, precision, recall, f1_score, mean_iou, dice_coefficient = compute_metrics(pred_mask, gt_mask, iou_threshold)
        
        pq_values.append(pq)
        precision_values.append(precision)
        recall_values.append(recall)
        f1_score_values.append(f1_score)
        mean_iou_values.append(mean_iou)
        dice_values.append(dice_coefficient)
        #print(mean_iou)
        
        if (i + 1) % 100 == 0:
            print(f"Processed {i + 1} files")
    
    pq_values = np.array(pq_values)
    precision_values = np.array(precision_values)
    recall_values = np.array(recall_values)
    f1_score_values = np.array(f1_score_values)
    mean_iou_values = np.array(mean_iou_values)
    dice_values = np.array(dice_values)
    
    metrics = {
        "mean_pq": np.mean(pq_values),
        "mean_precision": np.mean(precision_values),
        "mean_recall": np.mean(recall_values),
        "mean_f1_score": np.mean(f1_score_values),
        "mean_iou": np.mean(mean_iou_values),
        "mean_dice_coefficient": np.mean(dice_values)
    }
    
    return metrics, pq_values, precision_values, recall_values, f1_score_values, mean_iou_values, dice_values

# Function to save metrics to files
def save_metrics(metrics, pq_values, precision_values, recall_values, f1_score_values, mean_iou_values, dice_values, output_folder):
    # Create the output folder if it does not exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Save each array as a .npy file
    np.save(os.path.join(output_folder, 'pq_values.npy'), pq_values)
    np.save(os.path.join(output_folder, 'precision_values.npy'), precision_values)
    np.save(os.path.join(output_folder, 'recall_values.npy'), recall_values)
    np.save(os.path.join(output_folder, 'f1_score_values.npy'), f1_score_values)
    np.save(os.path.join(output_folder, 'mean_iou_values.npy'), mean_iou_values)
    np.save(os.path.join(output_folder, 'dice_values.npy'), dice_values)
    
    # Save the overall metrics as a text file
    with open(os.path.join(output_folder, 'overall_metrics.txt'), 'w') as f:
        for metric_name, metric_value in metrics.items():
            f.write(f"{metric_name}: {metric_value}\n")

# Function to process and save metrics
def process_and_save_metrics(pred_folder, gt_folder, output_folder):
    # Compute the mean PQ and log individual PQ values
    metrics, pq_values, precision_values, recall_values, f1_score_values, mean_iou_values, dice_values = process_folder(pred_folder, gt_folder, mask_need_transform=False)

    # Print all metrics
    for metric_name, metric_value in metrics.items():
        print(f"{metric_name}: {metric_value}")
    
    # Save the metrics to the specified folder
    save_metrics(metrics, pq_values, precision_values, recall_values, f1_score_values, mean_iou_values, dice_values, output_folder)
    print(f"Metrics and values saved to {output_folder}")


### Define paths to predicted masks and ground truth masks

In [ ]:
base_folder = '/home/icb/hanyi.zhang/public_organoid_datasets/OrganoID_dataset/testing'

gt_mask_dir = os.path.join(base_folder, 'segmentations_npy')
pred_mask_dir = os.path.join(base_folder, 'pred_mask')

# Specify the output folder for saving metrics
output_folder = os.path.join(base_folder, 'scratch_metrics_values_our')

process_and_save_metrics(pred_mask_dir, gt_mask_dir, output_folder)

# Tune threshold

In [ ]:
import os
# Define paths
base_path = '/home/icb/hanyi.zhang/public_organoid_datasets/OrganoID_dataset/testing_PDAC'

img_folder = base_path + '/images'
img_emb_folder = base_path + '/images_emb'

# Define confidence score threshold
confidence_score = np.arange(0.0, 1.05, 0.05)

for c_score in confidence_score:
    # Define paths to save the predicted masks and bboxes
    predictions_save_path = base_path + '/pred_mask_th_' + str(c_score)
    bboxes_save_path = base_path + '/pred_bbox_th_'  + str(c_score)

    os.makedirs(predictions_save_path, exist_ok=True)
    os.makedirs(bboxes_save_path, exist_ok=True)

    print(c_score)

    # Get list of files
    img_emb_files = os.listdir(img_emb_folder)

    # NMS threshold
    threshold = 0.5

    for img_emb_file in img_emb_files:
        file_idx = img_emb_file.rsplit('.', 1)[0]
        print(file_idx)

        # Define image path, label path and image embedding path
        image_path = os.path.join(img_folder, file_idx + '.png')
        image_embedding_path = os.path.join(img_emb_folder, img_emb_file)
        mask_save_path = os.path.join(predictions_save_path, file_idx + '.npy')
        bbox_save_path = os.path.join(bboxes_save_path, file_idx + '.npy')

        # Read the image
        image = Image.open(image_path)

        # Get the original size of the image
        original_size = image.size  # (width, height)
        original_size = (original_size[1], original_size[0])  # Convert to (height, width)

        # Load image embedding
        image_embedding = np.load(image_embedding_path)

        # Predict boxes and convert to original size
        #predicted_boxes, confidence_scores = evaluate_single_example(model, image_embedding)
        predicted_boxes, confidence_scores = evaluate_single_example_given_threshold(model, image_embedding, c_score)
        original_boxes_pred = inverse_boxes(convert_boxes(predicted_boxes) * 1024, original_size)
            
        kept_indices = non_max_suppression(original_boxes_pred, confidence_scores, threshold)
        if len(kept_indices) != 0:
            original_boxes_pred_filtered = original_boxes_pred[kept_indices]
            filtered_confidences = confidence_scores[kept_indices]
            
            # Predict masks
            image = cv2.imread(image_path)
            predictor.set_image(image)
            input_boxes = torch.tensor(original_boxes_pred_filtered).to(device)
            transformed_boxes = predictor.transform.apply_boxes_torch(input_boxes, image.shape[:2])
            masks, _, _ = predictor.predict_torch(
                point_coords=None,
                point_labels=None,
                boxes=transformed_boxes,
                multimask_output=False,
            )
            predicted_masks = masks.cpu().numpy()
            predicted_masks = transform_mask(predicted_masks)

            # Save the bounding boxes
            np.save(bbox_save_path, original_boxes_pred_filtered)
        else:
            # If no boxes, create an all-zero mask and save empty bounding boxes
            predicted_masks = np.zeros((original_size[0], original_size[1]), dtype=np.uint8)
            np.save(bbox_save_path, np.array([]))  # Save empty array for bounding boxes

        # Save the mask
        np.save(mask_save_path, predicted_masks)


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

# Base folder
base_folder = '/home/icb/hanyi.zhang/public_organoid_datasets/OrganoID_dataset/testing_PDAC'

# Define confidence score thresholds
confidence_scores = np.arange(0.0, 1.05, 0.05)

# Initialize lists to store mean values
pq = []
pre = []
recall = []
f1 = []
iou = []
dice = []

# Folder to save results
output_folder = os.path.join(base_folder, 'confidence_score_metrics')
os.makedirs(output_folder, exist_ok=True)

# Process each confidence score and record metrics
for c_score in confidence_scores:
    gt_mask_dir = os.path.join(base_folder, 'segmentations_npy')
    pred_mask_dir = os.path.join(base_folder, 'pred_mask_th_'  + str(c_score))
    
    # Get metrics from process_folder function
    metrics, pq_values, precision_values, recall_values, f1_score_values, mean_iou_values, dice_values = process_folder(
        pred_mask_dir, gt_mask_dir, mask_need_transform=False
    )
    
    # Record mean values for each metric
    pq.append(np.mean(pq_values))
    pre.append(np.mean(precision_values))
    recall.append(np.mean(recall_values))
    f1.append(np.mean(f1_score_values))
    iou.append(np.mean(mean_iou_values))
    dice.append(np.mean(dice_values))

# Save the lists to the output folder
np.save(os.path.join(output_folder, 'pq.npy'), pq)
np.save(os.path.join(output_folder, 'precision.npy'), pre)
np.save(os.path.join(output_folder, 'recall.npy'), recall)
np.save(os.path.join(output_folder, 'f1.npy'), f1)
np.save(os.path.join(output_folder, 'iou.npy'), iou)
np.save(os.path.join(output_folder, 'dice.npy'), dice)

# Plot F1 score vs. Confidence Score
plt.figure(figsize=(10, 6))
plt.plot(confidence_scores, f1, marker='o', linestyle='-', color='b')
plt.xlabel("Confidence Score")
plt.ylabel("Mean F1 Score")
plt.title("F1 Score vs. Confidence Score Threshold")
plt.grid(True)
plt.savefig(os.path.join(output_folder, 'f1_score_vs_confidence_score.png'))
plt.show()


In [ ]:
f1